# ETL SINIESTROS VIALES 
# Ciudad Autonoma de Buenos Aires CABA

**Por** 

**Data Analyst**

**GABRIEL GOMEZ RESTREPO**




Para  los Datasets  este  estudio sobre los accidentes viales de la Ciudad Autonoma de Buenos Aires(CABA), Argentina.
recurrimos a consultar  la siguientes links: 

**Para siniestros**

homicidios.xlsx   ( Hechos y Victimas)

https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales

**Para poblacion por Comuna** 

PBP_CO1025.xls   (poblacion por comuna  por edad y sexo)

proy_1025_depto_caba.xls   (resumen de PBP_CO1025, poblacion por comuna y por sexo)

https://data.buenosaires.gob.ar/dataset/estructura-poblacion


http://www.indec.mecon.ar/ftp/cuadros/poblacion

**Para Bariios por Comuna** 

comunas.xlsx  (Barrios por comuna)

https://data.buenosaires.gob.ar/dataset/comunas

La revision inicial de los datos que hemos otenido requiere de su estraccion tranformacion y  finalmente Cargar o crear una nueva base de datos usando nombres auque similares para  diferenciarlos con las fuentes originales.
## Librerias 
Las siguientes librerias son las que utilice. 


* **Pandas (import pandas as pd):**
Pandas es una biblioteca de Python fundamental en manejo de DataFrame que proporciona estructuras de datos flexibles y herramientas de análisis. 

* **NumPy (import numpy as np):**
NumPy es una biblioteca fundamental en Python para realizar operaciones numéricas.

* **%load_ext autoreload y %autoreload 2:**
Estos comandos son específicos de los cuadernos Jupyter y se utilizan para recargar automáticamente módulos antes de ejecutar una celda. %load_ext autoreload habilita la recarga automática de módulos, y %autoreload 2 configura la recarga automática para que sea más agresiva y recargue incluso las funciones de los módulos.

* **Warnings (import warnings):**
El módulo warnings proporciona herramientas para controlar las advertencias que emite Python. En este caso, se está configurando para ignorar las advertencias, lo cual puede ser útil para evitar que las advertencias llenen la salida de la consola y distraigan durante la ejecución del código.

In [170]:
import numpy as np
import pandas as pd


## Funciones 

* **Las funciones nos ahorran tiempo:**
>Esta son las funciones que utilice, generar unas funciones para agilizar nuestras tareas es muy importante evitando muchas acciones repetitivas ademas permiten un proceso mas claro y ordenado a continuacion las relaciono:  

## Funcion N°1 'ver_nulos'

In [171]:
def ver_nulos(df):
    # Creamos Diccionario para almacenar información que queremos ver
    dic = {"campo": [], "tipoDato": [], "%NOnull": [], "%null": [], "nulos": []}      
    for columna in df.columns:  # Iterar sobre todos los campo del DataFrame df
        # Calcular el porcentaje de No nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        # LLenamos  el Diccionario
        dic["campo"].append(columna)
        dic["tipoDato"].append(df[columna].apply(type).unique())
        dic["%NOnull"].append(round(porcentaje_no_nulos, 2))
        dic["%null"].append(round(100 - porcentaje_no_nulos, 2))
        dic["nulos"].append(df[columna].isnull().sum())             
    df_info = pd.DataFrame(dic) # Crearmos otro DataFrame con la información nueva 
    nulos = df.isnull().sum().sum() # Calcular los  nulos en el DataFrame      
    print(f'TOTAL NULOS  DataFrame: {nulos}')  # Imprimir el total de valores nulos
        # Mostrar los campos o columnas  con valores nulos y % 
    campo_con_nulos = df_info[df_info["nulos"] > 0][["campo", "%null"]].sort_values(by="%null", ascending=False)
    if not campo_con_nulos.empty:
        print("\nCAMPOS QUE TIENEN VALORES NULOS Y SU  % :")
        print(campo_con_nulos)    
    return df_info  # Retonamos el DataFrame con la información de cada columna


## Funcion N°2 'ver_filas_con_nulos'

**Crea un nuevo  DataFrame eligiendo una columna a la que desaamos ver, todos los nulos y esto permite  filtarar todas las columnas con el mismos'Id' esto es muy util para asociar que pasa con las celdas correspondientes. por ejemplo, ver  que los datos de otra columna podran ser utiles para llenar los nulos, o  considerar su importacia para par ver como una columna con pocoas datos, su eliminacion no, podria afentar las otras  .**

In [172]:


def ver_filas_con_nulos(df, columna):
    
    # Muestra un DataFrame con las filas donde la columna elegida   tiene valores nulos.
    # Parameters:
    # df (pd.DataFrame): El DataFrame original.   
    # columna (str): El nombre de la columna en la que buscar valores nulos.
    # Returns:
    # pd.DataFrame: Un nuevo DataFrame con las filas donde la columna especificada tiene valores nulos.        
    # Verificar que la columna especificada existe en el DataFrame
    
    
    if columna not in df.columns:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame")

    # Filtrar el DataFrame para obtener solo las filas donde la columna tiene valores nulos
    
    filas_con_nulos = df[df[columna].isnull()]
    
    return filas_con_nulos


# Llamar a la función y mostrar el resultado




# DATAFRAME df_hechos y df_victimas

**La idea iniocial es crear 2 Dataframe ('df_hechos' y df_victimas )  empercemos con :** 

## Creamos  df_hehos


In [173]:
df_hechos = pd.read_excel('./Datasets/Fuente/homicidios.xlsx', sheet_name='HECHOS')

**Miramos aspectos la informacion general del DataFrame y concluimos a primera vista que:** 

*Los  titulos de  columnas  algunas estan en mayusculas y otras en minusculas, hay nombres que podrian ser mas fasiles de entender por ejemplo seria mejos 'Año' que AAAA*  

*Tenemos la mayoria de las Columnas sin nulos (696) pero  TIPO_DE_CALLE, Calle, Altura, Cruce  si tienen presencia de nulos*

*Y tenemos los siguentes  tipos de datos  dtypes: datetime64[ns](1), float64(1), int64(5), object(14)*


In [174]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

**Empecemos cambiando los titulos de las columnas para mas fasilidad de entendimiento, ademmas usaremos la primera letra en mayusculas y las demas en minuscula, para mejor presentacion**

In [175]:
df_hechos.columns = df_hechos.columns.str.replace('_', ' ') # guiones por espacios
df_hechos.columns = [x.capitalize() for x in df_hechos.columns] #  mayúscula letra 1 y resto a minusculas
# Cambiamos algunos titulos 
df_hechos = df_hechos.rename(columns={'N victimas': 'Cantidad víctimas', 'Aaaa':'Año', 'Mm':'Mes', 'Dd':'Día',
                                      'Hh':'Hora entera', 'Xy (caba)':'XY (CABA)', 'Victima': 'Víctima'})

df_hechos.columns # oservando como quedan los titulares de las columnas

Index(['Id', 'Cantidad víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora entera', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Altura',
       'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x',
       'Pos y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

**An quedado los tutulos de las columnas  mayúscula letra 1 y resto a minusculas y colocamos algunos nombres mas amigables para todas las personas**

## Nulos
**Usuamos Funcion N°1 'ver_nulos' para  profundisar lo ya visto con  'df_hechos.info()'** 

In [176]:
def ver_nulos(df):
    # Creamos Diccionario para almacenar información que queremos ver
    dic = {"campo": [], "tipoDato": [], "%NOnull": [], "%null": [], "nulos": []}      
    for columna in df.columns:  # Iterar sobre todos los campo del DataFrame df
        # Calcular el porcentaje de No nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        # LLenamos  el Diccionario
        dic["campo"].append(columna)
        dic["tipoDato"].append(df[columna].apply(type).unique())
        dic["%NOnull"].append(round(porcentaje_no_nulos, 2))
        dic["%null"].append(round(100 - porcentaje_no_nulos, 2))
        dic["nulos"].append(df[columna].isnull().sum())             
    df_info = pd.DataFrame(dic) # Crearmos otro DataFrame con la información nueva 
    nulos = df.isnull().sum().sum() # Calcular los  nulos en el DataFrame      
    print(f'TOTAL NULOS  DataFrame: {nulos}')  # Imprimir el total de valores nulos
        # Mostrar los campos o columnas  con valores nulos y % 
    campo_con_nulos = df_info[df_info["nulos"] > 0][["campo", "%null"]].sort_values(by="%null", ascending=False)
    if not campo_con_nulos.empty:
        print("\nCAMPOS QUE TIENEN VALORES NULOS Y SU  % :")
        print(campo_con_nulos)    
    return df_info  # Retonamos el DataFrame con la información de cada columna


In [177]:
ver_nulos(df_hechos)

TOTAL NULOS  DataFrame: 747

CAMPOS QUE TIENEN VALORES NULOS Y SU  % :
                    campo  %null
11                 Altura  81.47
12                  Cruce  24.57
13  Dirección normalizada   1.15
10                  Calle   0.14


,campo,tipoDato,%NOnull,%null,nulos
0,Id,[<class 'str'>],100.00,0.00,0
1,Cantidad víctimas,[<class 'int'>],100.00,0.00,0
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.00,0.00,0
3,Año,[<class 'int'>],100.00,0.00,0
4,Mes,[<class 'int'>],100.00,0.00,0
5,Día,[<class 'int'>],100.00,0.00,0
6,Hora,"[<class 'datetime.time'>, <class 'str'>, <clas...",100.00,0.00,0
7,Hora entera,"[<class 'int'>, <class 'str'>]",100.00,0.00,0
8,Lugar del hecho,[<class 'str'>],100.00,0.00,0
9,Tipo de calle,[<class 'str'>],100.00,0.00,0


## Eliminar Columna 'Altura' 

Vamos a eliminar la columna 'Altura' ya que  tiene un porcentage muy alto de nulos (81.47%), y  su informacion la podemos conseguir en 

las columnas 'Direccion normalizada' y en 'Lugar del hecho'

In [178]:
# Eliminar la columna 'Altura' del DataFrame df_hechos.
df_hechos = df_hechos.drop('Altura', axis=1)

In [179]:
df_hechos.columns

Index(['Id', 'Cantidad víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora',
       'Hora entera', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x', 'Pos y',
       'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

**Confirmacion de nuevo encbezaso de columnas y eliminacion de 'Altura'**

## Usuamos Funcion N°2 'ver_filas_con_nulos'


**La columna 'Cruce'  tiene 171 nulos (24.57%), creamos un DataFrame(df_hechosCruce) para comparar sus nulos con   las columnas :  'Lugar del hecho', 'Tipo de calle', 'Calle', 'Dirección normalizada',  'XY (CABA)', 'Pos x', 'Pos y'** 

In [180]:
df_hechosCruce = ver_filas_con_nulos(df_hechos, 'Cruce')
df_hechosCruce

,Id,Cantidad víctimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11,AV ENTRE RIOS 1366,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO
14,2016-0020,1,2016-02-17,2016,2,17,16:00:00,16,SUIPACHA 156,CALLE,SUIPACHA,NaN,SUIPACHA 156,1,Point (107684.31807562 102519.17333314),-58.37952223,-34.60653282,PEATON-AUTO,PEATON,AUTO
33,2016-0047,1,2016-04-15,2016,4,15,05:30:00,5,LIMA 1483,CALLE,LIMA,NaN,LIMA 1483,1,Point (107412.19098904 100391.02557777),-58.38247061,-34.62571878,PEATON-PASAJEROS,PEATON,PASAJEROS
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,2021-0082,1,2021-11-06,2021,11,6,04:39:00,4,AU 25 DE MAYO KM7 (ALTURA PASCO),AUTOPISTA,AUTOPISTA 25 DE MAYO,NaN,AUTOPISTA 25 DE MAYO y PASCO,3,Point (106179.07517962 100523.29161457),-58.39591856,-34.62453459,MOTO-AUTO,MOTO,AUTO
685,2021-0087,1,2021-11-27,2021,11,27,11:22:00,11,ESMERALDA 1359,CALLE,ESMERALDA,NaN,ESMERALDA 1359,1,Point (107829.16426745 104166.84427133),-58.37795815,-34.59167894,PEATON-BICICLETA,PEATON,BICICLETA
687,2021-0089,1,2021-12-02,2021,12,2,01:10:00,1,AV. GAONA 3655,AVENIDA,GAONA AV.,NaN,GAONA AV. 3655,11,Point (98804.41713890 100872.30706871),-58.47633683,-34.62140594,MOTO-AUTO,MOTO,AUTO
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,"PERON, EVA AV.",NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


In [181]:
df_hechosCruce[[  'Lugar del hecho',  'Cruce', 'Dirección normalizada',   'Pos x', 'Pos y']]

,Lugar del hecho,Cruce,Dirección normalizada,Pos x,Pos y
2,AV ENTRE RIOS 2034,NaN,ENTRE RIOS AV. 2034,-58.39040293,-34.63189362
9,AV ENTRE RIOS 1366,NaN,ENTRE RIOS AV. 1366,-58.39114932,-34.62477387
14,SUIPACHA 156,NaN,SUIPACHA 156,-58.37952223,-34.60653282
33,LIMA 1483,NaN,LIMA 1483,-58.38247061,-34.62571878
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,-58.37714647568196,-34.63657525428238
...,...,...,...,...,...
680,AU 25 DE MAYO KM7 (ALTURA PASCO),NaN,AUTOPISTA 25 DE MAYO y PASCO,-58.39591856,-34.62453459
685,ESMERALDA 1359,NaN,ESMERALDA 1359,-58.37795815,-34.59167894
687,AV. GAONA 3655,NaN,GAONA AV. 3655,-58.47633683,-34.62140594
694,AV. EVA PERON 4071,NaN,"PERON, EVA AV. 4071",-58.47066794,-34.65021673


**Ipmutamos dataframe df_hechos  llenado los vacios  columna 'Cruces' con los elementos del mismo Id de la columna 'Lugar del hecho**

In [182]:

# Imputar los valores nulos de la columna 'Cruce' con los valores de la columna 'Lugar del hecho'
df_hechos['Cruce'].fillna(df_hechos['Lugar del hecho'], inplace=True)

print(df_hechos)


            Id  Cantidad víctimas      Fecha   Año  Mes  Día      Hora  \
0    2016-0001                  1 2016-01-01  2016    1    1  04:00:00   
1    2016-0002                  1 2016-01-02  2016    1    2  01:15:00   
2    2016-0003                  1 2016-01-03  2016    1    3  07:00:00   
3    2016-0004                  1 2016-01-10  2016    1   10  00:00:00   
4    2016-0005                  1 2016-01-21  2016    1   21  05:20:00   
..         ...                ...        ...   ...  ...  ...       ...   
691  2021-0093                  1 2021-12-13  2021   12   13  17:10:00   
692  2021-0094                  1 2021-12-20  2021   12   20  01:10:00   
693  2021-0095                  1 2021-12-30  2021   12   30  00:43:00   
694  2021-0096                  1 2021-12-15  2021   12   15  10:30:00   
695  2021-0097                  1 2021-11-18  2021   11   18  06:10:00   

    Hora entera                            Lugar del hecho Tipo de calle  \
0             4  AV PIEDRA BUENA Y 

In [183]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     696 non-null    object        
 1   Cantidad víctimas      696 non-null    int64         
 2   Fecha                  696 non-null    datetime64[ns]
 3   Año                    696 non-null    int64         
 4   Mes                    696 non-null    int64         
 5   Día                    696 non-null    int64         
 6   Hora                   696 non-null    object        
 7   Hora entera            696 non-null    object        
 8   Lugar del hecho        696 non-null    object        
 9   Tipo de calle          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Cruce                  696 non-null    object        
 12  Dirección normalizada  688 non-null    object        
 13  Comun

**Ya vemos que la columana 'Cruce' ya esta llena(696)  y non null**

## La columna  'Dirección normalizada' tiene 8 nulos (1.15%)

**Usuamos de nuevo la Funcion N°2 'ver_filas_con_nulos'**

**Creamos un dataframe(df_hechosDirN) para comparar los nulos de las columnas   'Calle', 'Cruce', 'Comuna', 'XY (CABA)', 'Pos x', 'Pos y'** 

In [185]:
df_hechosDirN = ver_filas_con_nulos(df_hechos, 'Dirección normalizada')

In [186]:
df_hechosDirN[[ 'Calle', 'Cruce', 'Dirección normalizada',  'XY (CABA)', 'Pos x', 'Pos y']]

,Calle,Cruce,Dirección normalizada,XY (CABA),Pos x,Pos y
38,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES PK 10000,NaN,Point (. .),.,.
106,AUTOPISTA BUENOS AIRES - LA PLATA,AU BUENOS AIRES - LA PLATA KM. 4,NaN,Point (. .),.,.
119,NaN,SD,NaN,Point (. .),.,.
180,AUTOPISTA PERITO MORENO,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,NaN,Point (. .),.,.
181,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AU DELLEPIANE 2400,NaN,Point (. .),.,.
313,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES KM 4.7,NaN,Point (. .),.,.
546,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV. KM 6,1",NaN,Point (. .),.,.
621,AUTOPISTA BUENOS AIRES - LA PLATA,"AU BUENOS AIRES LA PLATA KM 4,5",NaN,Point (. .),.,.


**Vemos los 8 nulos de 'Dirección normalizada' y observamops, puntos en 'XY (CABA)', 'Pos x', 'Pos y' ayi deberian haber numeros esto es muy importante**

**ya que es fundamental para la ubicion de los accidentes en el paño georefernciado, asique **

In [196]:
puntoY_count = df['Pos y'].value_counts()['.']
puntoX_count = df['Pos x'].value_counts()['.']
CabaXY_count = df['XY (CABA)'].value_counts()['Point (. .)']

print('Pos x:',   puntoX_count,   '  Pos y:',   puntoY_count,   '  XY (CABA):', CabaXY_count)


Pos x: 12   Pos y: 12   XY (CABA): 14


** Dado que 

In [168]:
df_hechos[[ 'Calle', 'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x', 'Pos y']][:5]

,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y
0,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022
1,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709
2,ENTRE RIOS AV.,AV ENTRE RIOS 2034,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362
3,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974
4,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630


In [166]:
puntoY_count = df['Pos y'].value_counts()['.']
puntoX_count = df['Pos x'].value_counts()['.']
CabaXY_count = df['XY (CABA)'].value_counts()['Point (. .)']
print(puntoY_count, puntoX_count, CabaXY_count)

12 12 14


In [167]:
df_hechosDirN = ver_filas_con_nulos(df_hechos, 'XY (CABA)')
df_hechos[[ 'Calle', 'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)', 'Pos x', 'Pos y']][:5]

,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y
0,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022
1,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709
2,ENTRE RIOS AV.,AV ENTRE RIOS 2034,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362
3,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974
4,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630
